# 特徴量生成汎用スクリプト

生データ読み込み

In [26]:
from typing import Any, Dict, Optional

import numpy as np
import pandas as pd
from sklearn.preprocessing import (
    LabelEncoder,
    MinMaxScaler,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
)

df = pd.read_csv('../notebooks/BQ.csv')
# df = pd.read_excel('../data/キリンシティ時間帯別売り上げ/23年5月~25年5月時間帯別売上.xlsx')

print(df.head())

   store_code store_name        date time_block  time   sales  customer_count  \
0         191  横浜ベイクォーター  2023-05-01          L    10       0               0   
1         191  横浜ベイクォーター  2023-05-01          L    11   67498              28   
2         191  横浜ベイクォーター  2023-05-01          L    12  111166              46   
3         191  横浜ベイクォーター  2023-05-01          L    13   67466              38   
4         191  横浜ベイクォーター  2023-05-01          L    14   41705              19   

   party_count  sales_target  prev_year_same_weekday_sales  \
0            0          2000                          3219   
1           14         74000                         76436   
2           21         79000                         81592   
3           22         82000                         84861   
4           12         42000                         43987   

   prev_year_same_date_sales  dow  
0                          0  Mon  
1                     115318  Mon  
2                      93527  Mo

データの基本情報を確認

In [ ]:
def load_data():
    global df, dff

    # dff が定義済み＆空じゃないならそれを使う
    if 'dff' in globals() and isinstance(dff, pd.DataFrame) and not dff.empty:
        print("既存の dff から読み込みます")
        return dff.copy()

    # まだ dff が空 or 定義なし → df をコピー
    if 'df' in globals() and isinstance(df, pd.DataFrame):
        print("df から読み込みます")
        dff = df.copy()
        return dff.copy()

    # 最後の手段としてファイルから読み込む
    try:
        df = pd.read_csv("your_data.csv", encoding="utf-8")
        dff = df.copy()
        print("ファイルから df を読み込み、dff にコピーしました")
        return dff.copy()
    except Exception as e:
        print("エラー: df も dff も存在せず、ファイル読み込みにも失敗しました。", e)
        return None

dff = load_data()
print(dff.head())


データ基本情報
データ形状: (9224, 12)
数値列 (8): ['store_code', 'time', 'sales', 'customer_count', 'party_count', 'sales_target', 'prev_year_same_weekday_sales', 'prev_year_same_date_sales']
カテゴリ列 (4): ['store_name', 'date', 'time_block', 'dow']
欠損値のある列 (0): []

=== 各列の詳細 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9224 entries, 0 to 9223
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   store_code                    9224 non-null   int64 
 1   store_name                    9224 non-null   object
 2   date                          9224 non-null   object
 3   time_block                    9224 non-null   object
 4   time                          9224 non-null   int64 
 5   sales                         9224 non-null   int64 
 6   customer_count                9224 non-null   int64 
 7   party_count                   9224 non-null   int64 
 8   sales_target                  9224 non-null  

過程状態の使い回し用

In [ ]:
dfff = dff
print(dfff.head())

数値列変換


In [ ]:
def transform_features(df: pd.DataFrame) -> pd.DataFrame:

    # 数値列を抽出
    numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]

    if not numeric_cols:
        print("数値列が見つかりません")
        return df

    print("\n=== 数値列一覧 ===")
    for i, col in enumerate(numeric_cols, 1):
        print(f"{i}. {col}")

    print("\n=== 数値列の統計情報 ===")
    print(df[numeric_cols].describe())

    # 変換する列を選択
    print("\n=== 列の選択 ===")
    print("変換したい列名をカンマ区切りで入力してください（例: column1,column2,column3）")
    print("全ての数値列を選択する場合は 'all' と入力してください")

    column_input = input("選択: ").strip()

    if column_input.lower() == 'all':
        selected_cols = numeric_cols
    else:
        try:
            column_names = [x.strip() for x in column_input.split(',')]
            selected_cols = [col for col in column_names if col in numeric_cols]
            invalid_cols = [col for col in column_names if col not in numeric_cols]
            if invalid_cols:
                print(f"警告: 以下の列名は数値列に存在しません: {', '.join(invalid_cols)}")
        except Exception:
            print("無効な入力です。処理を終了します。")
            return df

    if not selected_cols:
        print("有効な列が選択されませんでした")
        return df

    print(f"\n選択された列: {', '.join(selected_cols)}")

    # 変換方法の説明
    print("\n=== 変換方法一覧 ===")
    print("1. standard  - 標準化（平均0、標準偏差1）")
    print("2. log       - 対数変換（歪みを修正、正の値のみ）")
    print("3. minmax    - MinMax正規化（0-1スケール）")
    print("4. robust    - Robust Scaling（外れ値に頑健）")
    print("5. quantile  - Quantile変換（一様分布に変換）")
    print("6. binarize  - 二値化（閾値で0/1に変換）")
    print("7. skip      - 変換をスキップ")

    result_df = df.copy()
    transform_config = {}

    # 各列の変換方法を選択
    for col in selected_cols:
        print(f"\n--- 列 '{col}' の詳細情報 ---")
        col_data = df[col].dropna()
        print(f"最小値: {col_data.min():.3f}")
        print(f"最大値: {col_data.max():.3f}")
        print(f"平均値: {col_data.mean():.3f}")
        print(f"中央値: {col_data.median():.3f}")
        print(f"標準偏差: {col_data.std():.3f}")
        print(f"歪度: {col_data.skew():.3f}")
        print(f"欠損値数: {df[col].isnull().sum()}")

        if col_data.min() <= 0:
            print("⚠️ 注意: この列には0以下の値が含まれています（対数変換時は自動調整されます）")

        while True:
            method = input(f"'{col}' の変換方法を選択 (standard/log/minmax/robust/quantile/binarize/skip): ").strip().lower()

            if method in ['standard', 'log', 'minmax', 'robust', 'quantile', 'binarize', 'skip']:
                if method == 'binarize':
                    while True:
                        try:
                            threshold = float(input(f"'{col}' の閾値を入力（この値より大きい場合は1、以下は0）: ").strip())
                            transform_config[col] = {'method': method, 'threshold': threshold}
                            break
                        except ValueError:
                            print("数値を入力してください")
                elif method != 'skip':
                    transform_config[col] = method
                break
            else:
                print("無効な選択です。再度入力してください。")

    if not transform_config:
        print("変換する列が選択されませんでした")
        return df

    # 変換実行
    print("\n=== 変換実行 ===")

    for col, method_config in transform_config.items():
        try:
            # methodがstrの場合（従来の形式）とdictの場合（binarizeなど）を処理
            if isinstance(method_config, str):
                method = method_config
            else:
                method = method_config['method']

            if method == 'standard':
                scaler = StandardScaler()
                result_df[[col]] = scaler.fit_transform(result_df[[col]])
                print(f"✓ {col}: 標準化完了")

            elif method == 'log':
                # 負の値がある場合は自動的にシフト
                min_val = result_df[col].min()
                if min_val <= 0:
                    shift = 1 - min_val
                    result_df[col] = np.log(result_df[col] + shift)
                    print(f"✓ {col}: 対数変換完了（シフト量: {shift:.3f}）")
                else:
                    result_df[col] = np.log(result_df[col])
                    print(f"✓ {col}: 対数変換完了")

            elif method == 'minmax':
                scaler = MinMaxScaler()
                result_df[[col]] = scaler.fit_transform(result_df[[col]])
                print(f"✓ {col}: MinMax正規化完了")

            elif method == 'robust':
                scaler = RobustScaler()
                result_df[[col]] = scaler.fit_transform(result_df[[col]])
                print(f"✓ {col}: Robust Scaling完了")

            elif method == 'quantile':
                scaler = QuantileTransformer()
                result_df[[col]] = scaler.fit_transform(result_df[[col]])
                print(f"✓ {col}: Quantile変換完了")

            elif method == 'binarize':
                threshold = method_config['threshold']
                new_col = f"{col}_bin"
                result_df[new_col] = (result_df[col] > threshold).astype(int)
                print(f"✓ {col}: 二値化完了（閾値: {threshold}） -> {new_col}")

        except Exception as e:
            print(f"❌ {col}の変換中にエラーが発生しました: {e!s}")

    print("\n=== 変換完了 ===")
    print("変換後の統計情報:")

    # 変換された列と新しく作成された列を取得
    display_cols = []
    for col, method_config in transform_config.items():
        if isinstance(method_config, str):
            method = method_config
        else:
            method = method_config['method']

        if method == 'binarize':
            # 二値化の場合は新しい列を表示
            new_col = f"{col}_bin"
            if new_col in result_df.columns:
                display_cols.append(new_col)
        else:
            # その他の場合は元の列を表示
            display_cols.append(col)

    if display_cols:
        print(result_df[display_cols].describe())

    return result_df

dff = transform_features(dff)
# dff = transform_features(dfff)

print(dff.head())
print(f"\n変換後のデータ形状: {dff.shape}")


=== 数値列一覧 ===
1. store_code
2. time
3. sales
4. customer_count
5. party_count
6. sales_target
7. prev_year_same_weekday_sales
8. prev_year_same_date_sales

=== 数値列の統計情報 ===
       store_code         time          sales  customer_count  party_count  \
count      9224.0  9224.000000    9224.000000     9224.000000  9224.000000   
mean        191.0    16.255963   51565.131938       19.380312     8.672160   
std           0.0     3.715350   44825.553791       15.585010     6.700131   
min         191.0     0.000000       0.000000        0.000000     0.000000   
25%         191.0    13.000000   16621.500000        7.000000     3.000000   
50%         191.0    16.000000   40186.500000       16.000000     7.000000   
75%         191.0    19.000000   75516.250000       29.000000    13.000000   
max         191.0    23.000000  330946.000000       85.000000    33.000000   

        sales_target  prev_year_same_weekday_sales  prev_year_same_date_sales  
count    9224.000000                   9224

欠損値補完

In [8]:
def impute_missing_values(df: pd.DataFrame) -> pd.DataFrame:

    # 欠損値のある列を抽出
    missing_cols = df.columns[df.isna().any()].tolist()

    if not missing_cols:
        print("欠損値はありません")
        return df

    print("\n=== 欠損値のある列一覧 ===")
    for i, col in enumerate(missing_cols, 1):
        missing_count = df[col].isna().sum()
        missing_rate = (missing_count / len(df)) * 100
        col_type = "数値" if pd.api.types.is_numeric_dtype(df[col]) else "カテゴリカル"
        print(f"{i}. {col} - {missing_count}個 ({missing_rate:.1f}%) [{col_type}]")

    print("\n=== 欠損値詳細情報 ===")
    missing_info = df[missing_cols].isnull().sum()
    for col in missing_cols:
        total_missing = missing_info[col]
        missing_rate = (total_missing / len(df)) * 100
        print(f"{col}: {total_missing}/{len(df)} ({missing_rate:.1f}%)")

    # 補完する列を選択
    print("\n=== 列の選択 ===")
    print("補完したい列名をカンマ区切りで入力してください（例: age,income,category）")
    print("欠損値のある全ての列を選択する場合は 'all' と入力してください")

    column_input = input("選択: ").strip()

    if column_input.lower() == 'all':
        selected_cols = missing_cols
    else:
        try:
            column_names = [x.strip() for x in column_input.split(',')]
            selected_cols = [col for col in column_names if col in missing_cols]
            invalid_cols = [col for col in column_names if col not in missing_cols]
            if invalid_cols:
                print(f"警告: 以下の列名は欠損値のある列に存在しません: {', '.join(invalid_cols)}")
        except Exception:
            print("無効な入力です。処理を終了します。")
            return df

    if not selected_cols:
        print("有効な列が選択されませんでした")
        return df

    print(f"\n選択された列: {', '.join(selected_cols)}")

    # 補完方法の説明
    print("\n=== 補完方法一覧 ===")
    print("数値列で利用可能:")
    print("1. mean      - 平均値で補完")
    print("2. median    - 中央値で補完")
    print("3. mode      - 最頻値で補完")
    print("\n全ての列で利用可能:")
    print("4. forward   - 前方補完（前の値で埋める）")
    print("5. backward  - 後方補完（後の値で埋める）")
    print("6. fill      - 指定値で補完")
    print("7. drop      - 欠損値のある行を削除")
    print("8. skip      - 補完をスキップ")

    result_df = df.copy()
    impute_config = {}
    rows_to_drop = set()  # 削除対象の行インデックス

    # 各列の補完方法を選択
    for col in selected_cols:
        print(f"\n--- 列 '{col}' の詳細情報 ---")
        is_numeric = pd.api.types.is_numeric_dtype(df[col])
        missing_count = df[col].isna().sum()
        missing_rate = (missing_count / len(df)) * 100

        print(f"データ型: {'数値' if is_numeric else 'カテゴリカル'}")
        print(f"欠損値: {missing_count}個 ({missing_rate:.1f}%)")

        if is_numeric:
            non_missing = df[col].dropna()
            if len(non_missing) > 0:
                print(f"平均値: {non_missing.mean():.3f}")
                print(f"中央値: {non_missing.median():.3f}")
                if len(non_missing.mode()) > 0:
                    print(f"最頻値: {non_missing.mode().iloc[0]:.3f}")
        else:
            value_counts = df[col].value_counts()
            print(f"ユニーク値数: {len(value_counts)}")
            if len(value_counts) > 0:
                print(f"最頻値: {value_counts.index[0]}")
                print(f"上位3値: {dict(value_counts.head(3))}")

        # 利用可能な方法を表示
        available_methods = ['forward', 'backward', 'fill', 'drop', 'skip']
        if is_numeric:
            available_methods = ['mean', 'median', 'mode'] + available_methods
        else:
            available_methods = ['mode'] + available_methods

        print(f"利用可能な方法: {', '.join(available_methods)}")

        while True:
            method = input(f"'{col}' の補完方法を選択: ").strip().lower()

            if method == 'skip':
                break
            elif method in available_methods:
                if method == 'fill':
                    fill_value = input("補完する値を入力: ").strip()
                    # 数値列の場合は数値に変換を試行
                    if is_numeric:
                        try:
                            fill_value = float(fill_value)
                        except ValueError:
                            print("数値列には数値を入力してください。再度入力してください。")
                            continue
                    impute_config[col] = {'method': method, 'fill_value': fill_value}
                elif method == 'drop':
                    # 削除対象の行を記録
                    drop_indices = df[df[col].isna()].index
                    rows_to_drop.update(drop_indices)
                    print(f"列'{col}'の欠損値がある{len(drop_indices)}行を削除対象に追加")
                else:
                    impute_config[col] = {'method': method}
                break
            else:
                print("無効な選択です。再度入力してください。")

    if not impute_config and not rows_to_drop:
        print("補完する列が選択されませんでした")
        return df

    # 補完実行
    print("\n=== 補完実行 ===")

    # 行削除を先に実行
    if rows_to_drop:
        result_df = result_df.drop(index=list(rows_to_drop))
        print(f"✓ {len(rows_to_drop)}行を削除しました")

        # インデックスをリセット
        result_df = result_df.reset_index(drop=True)

    # 各列の補完を実行
    for col, config in impute_config.items():
        method = config['method']

        try:
            missing_before = result_df[col].isna().sum()

            if method == 'mean':
                fill_value = result_df[col].mean()
                result_df[col].fillna(fill_value, inplace=True)
                print(f"✓ {col}: 平均値補完完了 (補完値: {fill_value:.3f})")

            elif method == 'median':
                fill_value = result_df[col].median()
                result_df[col].fillna(fill_value, inplace=True)
                print(f"✓ {col}: 中央値補完完了 (補完値: {fill_value:.3f})")

            elif method == 'mode':
                mode_values = result_df[col].mode()
                if len(mode_values) > 0:
                    fill_value = mode_values.iloc[0]
                    result_df[col].fillna(fill_value, inplace=True)
                    print(f"✓ {col}: 最頻値補完完了 (補完値: {fill_value})")
                else:
                    print(f"❌ {col}: 最頻値が見つかりませんでした")

            elif method == 'forward':
                result_df[col].fillna(method='ffill', inplace=True)
                print(f"✓ {col}: 前方補完完了")

            elif method == 'backward':
                result_df[col].fillna(method='bfill', inplace=True)
                print(f"✓ {col}: 後方補完完了")

            elif method == 'fill':
                fill_value = config['fill_value']
                result_df[col].fillna(fill_value, inplace=True)
                print(f"✓ {col}: 指定値補完完了 (補完値: {fill_value})")

            missing_after = result_df[col].isna().sum()
            compensated = missing_before - missing_after
            if compensated > 0:
                print(f"  → {compensated}個の欠損値を補完")

        except Exception as e:
            print(f"❌ {col}の補完中にエラーが発生しました: {e!s}")

    print("\n=== 補完完了 ===")

    # 補完後の欠損値状況を確認
    remaining_missing = result_df.isnull().sum().sum()
    if remaining_missing > 0:
        print(f"残りの欠損値: {remaining_missing}個")
        remaining_cols = result_df.columns[result_df.isna().any()].tolist()
        print(f"欠損値が残っている列: {remaining_cols}")
    else:
        print("✓ 全ての欠損値が処理されました")

    return result_df

dff = impute_missing_values(dff)
# dff = impute_missing_values(dfff)

print(dff.head())
print(f"\n変換後のデータ形状: {dff.shape}")



カテゴリ変数のエンコーディング

In [24]:
def unified_encoding(df: pd.DataFrame,
                    encoding_config: Optional[Dict[str, Dict[str, Any]]] = None,
                    interactive: bool = True) -> pd.DataFrame:

    result_df = df.copy()
    categorical_cols = [col for col in df.columns if not pd.api.types.is_numeric_dtype(df[col])]

    if not categorical_cols:
        print("カテゴリカル列が見つかりません")
        return result_df

    if encoding_config is None and interactive:
        encoding_config = {}

        print("\n=== カテゴリカル列一覧 ===")
        for i, col in enumerate(categorical_cols, 1):
            print(f"{i}. {col}")

        print("\n=== カテゴリカル列の詳細情報 ===")
        for col in categorical_cols:
            unique_vals = df[col].unique()
            print(f"{col}: {len(unique_vals)}個のユニーク値 - {list(unique_vals)[:5]}{'...' if len(unique_vals) > 5 else ''}")

        column_input = input("変換したい列名をカンマ区切りで入力してください（未指定は Enter）: ").strip()

        if column_input.lower() == '':
            selected_cols = categorical_cols
        else:
            try:
                column_names = [x.strip() for x in column_input.split(',')]
                selected_cols = [col for col in column_names if col in categorical_cols]
                invalid_cols = [col for col in column_names if col not in categorical_cols]
                if invalid_cols:
                    print(f"警告: 以下の列名はカテゴリカル列に存在しません: {', '.join(invalid_cols)}")
            except Exception:
                print("無効な入力です。処理を終了します。")
                return result_df

        if not selected_cols:
            print("有効な列が選択されませんでした")
            return result_df

        print(f"\n選択された列: {', '.join(selected_cols)}")

        # エンコーディング方法の説明
        print("\n=== エンコーディング方法一覧 ===")
        print("1. onehot     - ワンホットエンコーディング（カテゴリ数が少ない場合）")
        print("2. label      - ラベルエンコーディング（順序関係がある場合）")
        print("3. frequency  - 頻度エンコーディング（出現頻度で数値化）")
        print("4. target     - ターゲットエンコーディング（ターゲット変数との関係を利用）")
        print("5. skip       - エンコーディングをスキップ")

        for col in selected_cols:
            print(f"\n--- 列 '{col}' の詳細情報 ---")
            unique_vals = df[col].unique()
            value_counts = df[col].value_counts()
            print(f"ユニーク値数: {len(unique_vals)}")
            print(f"欠損値数: {df[col].isnull().sum()}")
            print(f"上位5つの値: {dict(value_counts.head())}")

            if len(unique_vals) > 10:
                print("⚠️ 注意: ユニーク値が多いため、ワンホットエンコーディングは推奨されません")

            while True:
                method = input(f"'{col}' のエンコーディング方法を選択 (onehot/label/frequency/target/skip): ").strip().lower()

                if method == 'skip':
                    break
                elif method in ['onehot', 'label', 'frequency']:
                    encoding_config[col] = {'method': method}
                    break
                elif method == 'target':
                    target_col = input("ターゲット列名を入力: ").strip()
                    if target_col in df.columns:
                        encoding_config[col] = {'method': method, 'target_col': target_col}
                        break
                    else:
                        print(f"列'{target_col}'が見つかりません。再度入力してください。")
                else:
                    print("無効な選択です。再度入力してください。")

    if encoding_config is None:
        encoding_config = {}

    if not encoding_config:
        print("エンコーディングする列が選択されませんでした")
        return result_df

    # エンコーディング実行
    print("\n=== エンコーディング実行 ===")

    for col, config in encoding_config.items():
        if col not in categorical_cols:
            print(f"❌ 警告: 列'{col}'はカテゴリカル列ではありません")
            continue

        method = config.get('method', 'onehot')

        try:
            if method == 'onehot':
                # ワンホットエンコーディング
                dummies = pd.get_dummies(result_df[col], prefix=col, drop_first=True)
                result_df = pd.concat([result_df.drop(col, axis=1), dummies], axis=1)
                print(f"✓ {col}: ワンホットエンコーディング完了 -> {len(dummies.columns)}個の新しい特徴量")

            elif method == 'label':
                # ラベルエンコーディング
                le = LabelEncoder()
                result_df[col] = le.fit_transform(result_df[col])
                print(f"✓ {col}: ラベルエンコーディング完了")

            elif method == 'frequency':
                # 頻度エンコーディング
                freq = result_df[col].value_counts(normalize=True)
                new_col = f"{col}_freq"
                result_df[new_col] = result_df[col].map(freq)
                print(f"✓ {col}: 頻度エンコーディング完了 -> {new_col}")

            elif method == 'target':
                # ターゲットエンコーディング
                target_col = config.get('target_col')
                if target_col and target_col in result_df.columns:
                    target_mean = result_df.groupby(col)[target_col].mean()
                    new_col = f"{col}_target"
                    result_df[new_col] = result_df[col].map(target_mean)
                    print(f"✓ {col}: ターゲットエンコーディング完了 -> {new_col}")
                else:
                    print(f"❌ ターゲット列'{target_col}'が見つかりません。列'{col}'をスキップ")

        except Exception as e:
            print(f"❌ {col}のエンコーディング中にエラーが発生しました: {e!s}")

    print("\n=== エンコーディング完了 ===")

    return result_df

dff = unified_encoding(dff)
# dff = unified_encoding(dfff)

print(dff.head())
print(f"\n変換後のデータ形状: {dff.shape}")


=== カテゴリカル列一覧 ===
1. store_name
2. date
3. time_block
4. dow

=== カテゴリカル列の詳細情報 ===
store_name: 1個のユニーク値 - ['横浜ベイクォーター']
date: 762個のユニーク値 - ['2023-05-01', '2023-05-02', '2023-05-03', '2023-05-04', '2023-05-05']...
time_block: 3個のユニーク値 - ['L', 'D', '未使用']
dow: 7個のユニーク値 - ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']...

選択された列: time_block

=== エンコーディング方法一覧 ===
1. onehot     - ワンホットエンコーディング（カテゴリ数が少ない場合）
2. label      - ラベルエンコーディング（順序関係がある場合）
3. frequency  - 頻度エンコーディング（出現頻度で数値化）
4. target     - ターゲットエンコーディング（ターゲット変数との関係を利用）
5. skip       - エンコーディングをスキップ

--- 列 'time_block' の詳細情報 ---
ユニーク値数: 3
欠損値数: 0
上位5つの値: {'L': np.int64(5383), 'D': np.int64(3724), '未使用': np.int64(117)}

=== エンコーディング実行 ===
✓ time_block: ワンホットエンコーディング完了 -> 2個の新しい特徴量

=== エンコーディング完了 ===
変換後のDataFrame形状: (9224, 13)
   store_code store_name        date  time     sales  customer_count  \
0         191  横浜ベイクォーター  2023-05-01    10 -1.150414               0   
1         191  横浜ベイクォーター  2023-05-01    11  0.355461              28   
2    

多項式特徴量の生成

In [10]:
def polynomial_features(df: pd.DataFrame) -> pd.DataFrame:
    # 数値列を抽出
    numeric_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]

    if len(numeric_cols) < 2:
        print("多項式特徴量の生成には2つ以上の数値列が必要です")
        return df

    print("\n=== 数値列一覧 ===")
    for i, col in enumerate(numeric_cols, 1):
        print(f"{i}. {col}")

    print("\n=== 数値列の統計情報 ===")
    print(df[numeric_cols].describe())

    # 変換する列を選択
    print("\n=== 列の選択 ===")
    print("多項式特徴量に使用したい列名をカンマ区切りで入力してください（例: age,income,score）")
    print("全ての数値列を選択する場合は 'all' と入力してください")
    print("※注意: 列数が多いと特徴量数が爆発的に増加します")

    column_input = input("選択: ").strip()

    if column_input.lower() == 'all':
        selected_cols = numeric_cols
    else:
        try:
            column_names = [x.strip() for x in column_input.split(',')]
            selected_cols = [col for col in column_names if col in numeric_cols]
            invalid_cols = [col for col in column_names if col not in numeric_cols]
            if invalid_cols:
                print(f"警告: 以下の列名は数値列に存在しません: {', '.join(invalid_cols)}")
        except Exception:
            print("無効な入力です。処理を終了します。")
            return df

    if len(selected_cols) < 2:
        print("2つ以上の有効な数値列が必要です")
        return df

    print(f"\n選択された列: {', '.join(selected_cols)}")

    # 各列の詳細情報表示
    print("\n=== 選択列の詳細情報 ===")
    for col in selected_cols:
        col_data = df[col].dropna()
        print(f"{col}: 範囲[{col_data.min():.2f}, {col_data.max():.2f}], 平均={col_data.mean():.2f}, 標準偏差={col_data.std():.2f}")

    # 予想される特徴量数を計算して表示
    n_cols = len(selected_cols)
    print("\n=== 多項式特徴量設定 ===")
    print("次数による特徴量数の予想:")
    print(f"1次: {n_cols}個 (元の特徴量)")

    import math
    degree_2_count = n_cols + math.comb(n_cols, 2) + n_cols  # 1次 + 交互作用 + 2乗
    degree_3_count = sum(math.comb(n_cols + r - 1, r) for r in range(1, 4))  # 1次〜3次の組み合わせ

    print(f"2次: 約{degree_2_count}個")
    if n_cols <= 5:  # 5列以下の場合のみ3次を表示
        print(f"3次: 約{degree_3_count}個")

    if n_cols > 5:
        print("⚠️ 警告: 列数が多いため、2次までの使用を推奨します")

    # 次数の選択
    while True:
        try:
            degree = int(input("多項式の次数を選択 (2または3): ").strip())
            if degree in [2, 3]:
                break
            else:
                print("2または3を入力してください")
        except ValueError:
            print("数値を入力してください")

    # 交互作用のみかどうか
    print("\n多項式特徴量のタイプ:")
    print("1. full       - すべての項（x², xy, x³など）")
    print("2. interaction - 交互作用のみ（xy, xyzなど、単一変数の累乗は除く）")

    while True:
        interaction_type = input("タイプを選択 (full/interaction): ").strip().lower()
        if interaction_type in ['full', 'interaction']:
            interaction_only = (interaction_type == 'interaction')
            break
        else:
            print("fullまたはinteractionを入力してください")

    # バイアス項（定数項）の追加
    while True:
        bias_input = input("バイアス項（定数項）を追加しますか？ (y/n): ").strip().lower()
        if bias_input in ['y', 'yes', 'n', 'no']:
            include_bias = bias_input in ['y', 'yes']
            break
        else:
            print("yまたはnを入力してください")

    # 多項式特徴量生成
    print("\n=== 多項式特徴量生成実行 ===")

    try:
        from sklearn.preprocessing import PolynomialFeatures

        poly = PolynomialFeatures(
            degree=degree,
            interaction_only=interaction_only,
            include_bias=include_bias
        )

        print(f"設定: 次数={degree}, 交互作用のみ={interaction_only}, バイアス項={include_bias}")

        # 選択された列のみで多項式特徴量を生成
        poly_features = poly.fit_transform(df[selected_cols])

        # 特徴量名を生成
        feature_names = poly.get_feature_names_out(selected_cols)

        # 元の列を除いた新しい特徴量のみを取得
        if not include_bias:
            # 元の列（最初のn_cols個）を除く
            new_features = poly_features[:, n_cols:]
            new_feature_names = feature_names[n_cols:]
        else:
            # バイアス項があるので、最初の1個とその次のn_cols個を除く
            new_features = poly_features[:, n_cols + 1:]
            new_feature_names = feature_names[n_cols + 1:]

            # バイアス項を別途追加
            bias_features = poly_features[:, 0:1]
            bias_names = feature_names[0:1]

        # 新しい特徴量をDataFrameに変換
        if len(new_feature_names) > 0:
            new_poly_df = pd.DataFrame(new_features, columns=new_feature_names, index=df.index)
            result_df = pd.concat([df, new_poly_df], axis=1)
        else:
            result_df = df.copy()

        # バイアス項がある場合は追加
        if include_bias:
            bias_df = pd.DataFrame(bias_features, columns=bias_names, index=df.index)
            result_df = pd.concat([result_df, bias_df], axis=1)

        print("✓ 多項式特徴量生成完了")
        print(f"✓ 元の列数: {len(selected_cols)}")
        print(f"✓ 生成された新しい特徴量数: {len(feature_names) - len(selected_cols) - (1 if include_bias else 0)}")
        if include_bias:
            print("✓ バイアス項: 1個")
        print(f"✓ 総特徴量数: {len(feature_names)}")

        print("\n生成された特徴量の例:")
        new_cols = [col for col in result_df.columns if col not in df.columns]
        for col in new_cols[:5]:  # 最初の5個だけ表示
            print(f"  - {col}")
        if len(new_cols) > 5:
            print(f"  ... 他{len(new_cols) - 5}個")

    except Exception as e:
        print(f"❌ 多項式特徴量生成中にエラーが発生しました: {e!s}")
        return df

    print("\n=== 多項式特徴量生成完了 ===")

    return result_df

dff = polynomial_features(dff)
# dff = polynomial_features(dfff)

print(dff.head())
print(f"\n変換後のデータ形状: {dff.shape}")

変数dffをcsvファイルとして保存

1. 保存したいcsvファイル名(.csv除く)を入力

2. 00_pretreatment.ipynbと同一のディレクトリに保存

In [ ]:
def save_to_csv(df, filename=None):

    if filename is None:
        filename = input("保存するファイル名を入力してください（.csvを除く）: ").strip()

    # 入力が空の場合のデフォルト値
    if not filename:
        filename = "output_data"
        print(f"⚠ ファイル名が指定されなかったため、デフォルト名 '{filename}.csv' を使用します")

    if not filename.endswith('.csv'):
        filename += '.csv'

    try:
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✓ CSVファイル保存完了: {filename}")
        print(f"  データ行数: {len(df)}")
        print(f"  列数: {len(df.columns)}")

        # ファイルサイズを表示
        import os
        if os.path.exists(filename):
            file_size = os.path.getsize(filename)
            print(f"  ファイルサイズ: {file_size:,} バイト")

        return filename

    except Exception as e:
        print(f"✗ 保存エラー: {e}")
        return None

if __name__ == "__main__":
    saved_file = save_to_csv(dff)

    if saved_file:
        print(f"\n🎉 ファイルが正常に保存されました: {saved_file}")
    else:
        print("\n❌ ファイルの保存に失敗しました")

csv保存後のdff,dfffを空にする

In [ ]:
if 'dff' in globals():
    del dff

In [ ]:
if 'dff' in globals():
    del dfff

一時確認用

In [25]:
try:
    print(dff.head())
except:
    print(dfff.head())

    store_code store_name        date  time     sales  customer_count  \
0          191  横浜ベイクォーター  2023-05-01    10 -1.150414               0   
1          191  横浜ベイクォーター  2023-05-01    11  0.355461              28   
2          191  横浜ベイクォーター  2023-05-01    12  1.329690              46   
3          191  横浜ベイクォーター  2023-05-01    13  0.354747              38   
4          191  横浜ベイクォーター  2023-05-01    14 -0.219979              19   
..         ...        ...         ...   ...       ...             ...   
95         191  横浜ベイクォーター  2023-05-08    20 -1.150414               0   
96         191  横浜ベイクォーター  2023-05-08    21 -0.975169               3   
97         191  横浜ベイクォーター  2023-05-08    22 -1.150414               0   
98         191  横浜ベイクォーター  2023-05-09    11 -0.206972              23   
99         191  横浜ベイクォーター  2023-05-09    12 -0.085584              27   

    party_count  sales_target  prev_year_same_weekday_sales  \
0             0          2000                          3219 